# 비지도학습 감성분석 - Lexicon 기반

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### < Wordnet Synset 및 Sentiwordnet SentiSynset 클래스 >

- 워드넷(Wordnet) : 영어의 의미 어휘목록
    - 워드넷은 영어 단어를 'synset'이라는 유의어 집단으로 분류하여 간략하고 일반적인 정의를 제공하고, 이러한 어휘목록 사이의 다양한 의미 관계를 기록한다.

- 워드넷 synset
    ``` 
        - synset.name() : 단어 의미 목록(명) 
            - <ex> present.n.01
        - synset.definition() : 정의
        - synset.pos() : 품사
        - synset.example() : 예제 문장
    ```

※ 총 **5가지의 품사**(Part-of-speech, POS)가 존재한다.
```    
- n : noun                  (명사)
- v : verb                  (동사)
- a : adjective             (형용사)
- s : adjective satellite   (반의어가 존재하지 않는 단어)
- r : adverb                (부사)
```

- Sentiwordnet
    - Wordnet과 굉장히 유사한데, 감정 지수와 객관성 지수를 가지고 있다. 
    - 감정 지수는 긍정 / 부정으로 나뉘고, 객관성 지수는 감성적이지 않으면 1이 된다.

### 1. Wordnet Synset

In [2]:
from nltk.corpus import wordnet # wordnet : 어휘사전

term = 'present'
synsets = wordnet.synsets(term) # present 단어 의미 목록 도출

##### wordnet 계속 입력하기 힘드므로, 처음 패키지 임포트 시 `from nltk.corpus import wordnet as wn` 으로 해도 된다.

In [3]:
# present 단어 의미 목록 형태 / 단어의 의미 갯수
type(synsets), len(synsets) 

(list, 18)

In [4]:
print(synsets) # synsets : 단어의 모든 의미들 - 객체들의 리스트

[Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [7]:
for synset in synsets[:5]:
    print(f'---- Name: {synset.name()} ----') # 단어 의미 목록 _ 단어.품사.번호
    print('POS: ', synset.lexname())    # 단어의 품사.목록
    print('정의: ', synset.definition())    # 단어의 정의
    print('표제어: ', synset.lemma_names()) # 동의어 단어 집합

---- Name: present.n.01 ----
POS:  noun.time
정의:  the period of time that is happening now; any continuous stretch of time including the moment of speech
표제어:  ['present', 'nowadays']
---- Name: present.n.02 ----
POS:  noun.possession
정의:  something presented as a gift
표제어:  ['present']
---- Name: present.n.03 ----
POS:  noun.communication
정의:  a verb tense that expresses actions or states at the time of speaking
표제어:  ['present', 'present_tense']
---- Name: show.v.01 ----
POS:  verb.perception
정의:  give an exhibition of to an interested audience
표제어:  ['show', 'demo', 'exhibit', 'present', 'demonstrate']
---- Name: present.v.02 ----
POS:  verb.communication
정의:  bring forward and present to the mind
표제어:  ['present', 'represent', 'lay_out']


- 어휘간의 유사도

In [8]:
for synset in wordnet.synsets('tiger'):
    print(synset.name(), synset.definition())

tiger.n.01 a fierce or audacious person
tiger.n.02 large feline of forests in most of Asia having a tawny coat with black stripes; endangered


In [9]:
# 단어, 품사를 아는 경우에는 synset()
tiger = wordnet.synset('tiger.n.02')
tree = wordnet.synset('tree.n.01')
lion = wordnet.synset('lion.n.01')
cat = wordnet.synset('cat.n.01')
dog = wordnet.synset('dog.n.01')

In [11]:
# 단어간의 유사도 - 단어.path_similarity(비교할 단어)
# wordnet.synset() 객체로 만든 단어를 입력해야 함
tiger.path_similarity(lion), tiger.path_similarity(dog), tiger.path_similarity(tree)

(0.3333333333333333, 0.16666666666666666, 0.07142857142857142)

In [12]:
# 5개 단어간의 유사도
similarities = []
entities = [tree, lion, tiger, cat, dog]
for entity in entities:
    similarity = [entity.path_similarity(another) for another in entities]
    similarities.append(similarity)

In [103]:
df = pd.DataFrame(similarities, columns=['tree', 'lion', 'tiger', 'cat', 'dog'])
df

,tree,lion,tiger,cat,dog
0,1.000000,0.071429,0.071429,0.076923,0.125000
1,0.071429,1.000000,0.333333,0.250000,0.166667
2,0.071429,0.333333,1.000000,0.250000,0.166667
3,0.076923,0.250000,0.250000,1.000000,0.200000
4,0.125000,0.166667,0.166667,0.200000,1.000000


##### ※ 인덱스 변경 : `set_index(keys=[k1, k2, ...], inplace=True/False, drop=True/False)`

In [104]:
# 유사도 비교 용이하게 하기 위해 인덱스 변경
index = ['tree', 'lion', 'tiger', 'cat', 'dog']
df.set_index(keys=[index], inplace=True)
df

,tree,lion,tiger,cat,dog
tree,1.000000,0.071429,0.071429,0.076923,0.125000
lion,0.071429,1.000000,0.333333,0.250000,0.166667
tiger,0.071429,0.333333,1.000000,0.250000,0.166667
cat,0.076923,0.250000,0.250000,1.000000,0.200000
dog,0.125000,0.166667,0.166667,0.200000,1.000000


### 2. Sentiwordnet SentiSynset

- SentiSynset 객체

In [29]:
from nltk.corpus import sentiwordnet

senti_synsets = list(sentiwordnet.senti_synsets('slow'))

##### sentiwordnet 계속 입력하기 힘드므로, 처음 패키지 임포트 시 `from nltk.corpus import sentiwordnet as sw` 으로 해도 된다.

In [30]:
print(type(senti_synsets))
print(len(senti_synsets))
print(senti_synsets)

<class 'list'>
11
[SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [31]:
senti_synsets = list(sentiwordnet.senti_synsets('father'))
print(type(senti_synsets))
print(len(senti_synsets))
print(senti_synsets)

<class 'list'>
9
[SentiSynset('father.n.01'), SentiSynset('forefather.n.01'), SentiSynset('father.n.03'), SentiSynset('church_father.n.01'), SentiSynset('father.n.05'), SentiSynset('father.n.06'), SentiSynset('founder.n.02'), SentiSynset('don.n.03'), SentiSynset('beget.v.01')]


In [32]:
# father 단어의 긍정감성 지수, 부정감성 지수, 객관성 지수
# 긍정도 부정도 아닌 객관성을 지닌 단어
father = sentiwordnet.senti_synset('father.n.01')
father.pos_score(), father.neg_score(), father.obj_score()

(0.0, 0.0, 1.0)

In [33]:
# mother 단어의 긍정감성 지수, 부정감성 지수, 객관성 지수
# 긍정도 부정도 아닌 객관성을 지닌 단어
mother = sentiwordnet.senti_synset('mother.n.01')
mother.pos_score(), mother.neg_score(), mother.obj_score()

(0.0, 0.0, 1.0)

In [35]:
# fabulous 단어의 긍정감성 지수, 부정감성 지수, 객관성 지수
# 의미 : 엄청난, 굉장한 --> 긍정감성 지수 (0.875-0.125=0.75)
fabulous = sentiwordnet.senti_synset('fabulous.a.01')
fabulous.pos_score(), fabulous.neg_score(), fabulous.obj_score()

(0.875, 0.125, 0.0)

In [37]:
list(sentiwordnet.senti_synsets('just'))

[SentiSynset('just.a.01'),
 SentiSynset('equitable.a.01'),
 SentiSynset('fair.a.01'),
 SentiSynset('good.s.07'),
 SentiSynset('merely.r.01'),
 SentiSynset('precisely.r.01'),
 SentiSynset('just.r.03'),
 SentiSynset('just.r.04'),
 SentiSynset('barely.r.01'),
 SentiSynset('just.r.06')]

In [40]:
# precisely 단어의 긍정감성 지수, 부정감성 지수, 객관성 지수
precisely = sentiwordnet.senti_synset('precisely.r.01')
precisely.pos_score(), precisely.neg_score(), precisely.obj_score()

(0.125, 0.0, 0.875)

In [41]:
# work 단어의 긍정감성 지수, 부정감성 지수, 객관성 지수
work = sentiwordnet.senti_synset('work.v.01')
work.pos_score(), work.neg_score(), work.obj_score()

(0.0, 0.0, 1.0)

In [42]:
# hit 단어의 긍정감성 지수, 부정감성 지수, 객관성 지수
hit = sentiwordnet.senti_synset('hit.v.01')
hit.pos_score(), hit.neg_score(), hit.obj_score()

(0.0, 0.0, 1.0)

In [43]:
wordnet.NOUN, wordnet.ADJ, wordnet.ADV, wordnet.VERB

('n', 'a', 'r', 'v')

- 감성지수 계산

In [46]:
from nltk import word_tokenize, pos_tag # 단어 토큰화, 단어 품사 태그

sentence = "It's good to see you again."
word_list = word_tokenize(sentence) # 단어 토큰화
word_list

['It', "'s", 'good', 'to', 'see', 'you', 'again', '.']

In [47]:
pos_tag(word_list)  # 단어 품사 태그 # 튜플 형태

[('It', 'PRP'),
 ("'s", 'VBZ'),
 ('good', 'JJ'),
 ('to', 'TO'),
 ('see', 'VB'),
 ('you', 'PRP'),
 ('again', 'RB'),
 ('.', '.')]

In [50]:
tag = ('good', 'JJ')
tag[1].startswith('J') # str.startswith(str or tuple) --> return: True/False - 지정한 문자열로 시작하면 T 아니면 F

True

In [52]:
# pos_tag -> wordnet 으로 바꿔주는 함수
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    return None    

In [53]:
for word, tag in pos_tag(word_list):
    print(word, tag)

It PRP
's VBZ
good JJ
to TO
see VB
you PRP
again RB
. .


In [54]:
# pos_tag -> wordnet 으로 바꿔주는 함수 사용
for word, tag in pos_tag(word_list):
    print(word, penn_to_wn(tag))

It None
's v
good a
to None
see v
you None
again r
. None


In [108]:
# Sentence 로부터 Senti_Synset 객체를 만드는 과정
sentence = "It's good to see you again."
word_list = [word for word in word_tokenize(sentence) if len(word) > 2] # 단어의 길이가 2 이상인 것들만 토큰화
word_list

['good', 'see', 'you', 'again']

In [109]:
for word, tag in pos_tag(word_list):
    wn_tag = penn_to_wn(tag) # pos_tag -> wordnet으로 바꾸는 함수 사용해서 wn_tag로 표시
    if wn_tag:
        synsets = list(sentiwordnet.senti_synsets(word, wn_tag))    # 단어 의미 목록을 synsets 변수로 받기
        synset = synsets[0] # 단어 의미 목록 중 첫 번째 거 synset 변수로 받기
        print(synset)

<good.a.01: PosScore=0.75 NegScore=0.0>
<see.n.01: PosScore=0.0 NegScore=0.0>
<again.r.01: PosScore=0.0 NegScore=0.0>


In [62]:
sentiment = 0. # 감성지수 초기값 0으로 설정해서 이후 과정에서 도출되는 수치들의 계산 결과에 영향미치지 않도록 함
for word, tag in pos_tag(word_list):
    wn_tag = penn_to_wn(tag)
    if wn_tag:
        synsets = list(sentiwordnet.senti_synsets(word, wn_tag))
        synset = synsets[0]
        sentiment += synset.pos_score() - synset.neg_score()
sentiment

0.75

In [65]:
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [66]:
sentiment = 0.
for word, tag in pos_tag(word_list):
    wn_tag = penn_to_wn(tag)
    if wn_tag:
        lemma = lemmatizer.lemmatize(word, wn_tag)  # 품사를 입력하여 표제어 추출기가 보다 정확한 결과를 도출하도록 함
        synsets = list(sentiwordnet.senti_synsets(lemma, wn_tag)) # lemma : 표제어 (본래 단어 형태)
        synset = synsets[0]
        sentiment += synset.pos_score() - synset.neg_score()
sentiment

0.75

In [71]:
from nltk import sent_tokenize
document = "I watched this video at a friend's house. I'm glad I did not waste money buying this one. The video cover has a scene from the 1975 movie Capricorn One. The movie starts out with several clips of rocket blow-ups, most not related to manned flight. Sibrel's smoking gun is a short video clip of the astronauts preparing a video broadcast. He edits in his own voice-over instead of letting us listen to what the crew had to say. The video curiously ends with a showing of the Zapruder film. His claims about radiation, shielding, star photography, and others lead me to believe is he extremely ignorant or has some sort of ax to grind against NASA, the astronauts, or American in general. His science is bad, and so is this video."

In [75]:
sentiment = 0.0
for sentence in sent_tokenize(document):
    word_list = [word for word in word_tokenize(sentence) if len(word) > 2]
    for word, tag in pos_tag(word_list):
        wn_tag = penn_to_wn(tag)
        if wn_tag:
            lemma = lemmatizer.lemmatize(word, wn_tag)
            synsets = list(sentiwordnet.senti_synsets(lemma, wn_tag))
            if not synsets:
                print(word)
                continue
            synset = synsets[0]
            sentiment += synset.pos_score() - synset.neg_score()
print('긍정' if sentiment >= 0 else '부정')

scene
blow-ups
Sibrel
voice-over
Zapruder
others
부정


- 감성을 계산해주는 함수 만들기

In [76]:
def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()        # 표제어 추출기
    raw_sentences = sent_tokenize(text)     # 문장 토큰화
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        word_list = [word for word in word_tokenize(raw_sentence) if len(word) > 2]     # 단어 토큰화
        tagged_sentence = pos_tag(word_list)    # (토큰화된 단어, 단어 품사) - 튜플 형태
        for word, tag in tagged_sentence:
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wordnet.NOUN, wordnet.ADJ, wordnet.ADV, wordnet.VERB):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma: # lemma == None 이면, 다시 루프 실행
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wordnet.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = sentiwordnet.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1   # 토큰화 과정이 일어난다면 +1 해준다. 
    
    if not tokens_count:    # 토큰화 과정이 전혀 일어나지 않으면, 즉 입력된 문장의 단어 토큰화가 전혀 일어나지 않은 경우
        return 0            # 0을 반환 -> 결과 : 부정
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    return 1 if sentiment >= 0 else 0

### - IMDB 영화평 감성분석

In [87]:
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t', quoting=3)      # 3 : QUOTE-None == 인용구(큰따옴표) 무시
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- 데이터 전처리

In [88]:
# <br /> 태그 → 공백으로 변환
df.review = df.review.str.replace("<br />", " ")

# 구둣점, 숫자 제거 - 영문자가 아닌 것들 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()

In [79]:
df.shape

(25000, 3)

In [80]:
# 1000개 리뷰 분석
# df = df.iloc[:1000, :]
# df.shape

(1000, 3)

In [89]:
%time df['pred'] = df.review.apply(lambda x : swn_polarity(x))  # 위에서 만든 감성지수 계산하는 함수 사용해서 긍정/부정 예측

Wall time: 5min 18s


In [90]:
from sklearn.metrics import accuracy_score  # 실제 감성지수(sentiment)와 예측값(pred) 비교 -> 정확도 도출
accuracy_score(df.sentiment, df.pred)

0.62568

### < VADER Lexicon을 이용한 감성 분석 >

In [83]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer # 클래스

senti_analyzer = SentimentIntensityAnalyzer()   # 객체 생성
senti_score = senti_analyzer.polarity_scores(df.review[0])  # polarity_scores() : -1 ~ 1 사잇값
senti_score

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}

In [91]:
def vader_polarity(document, threshold=0.1):            # threshold 값 0.1  -> threshold(임계값) : 긍정과 부정을 나누는 기준점
    score = senti_analyzer.polarity_scores(document)
    return 1 if score['compound'] >= threshold else 0

In [92]:
%time df['vader_pred'] = df.review.apply(lambda x : vader_polarity(x, 0.1))   

Wall time: 58.1 s


In [93]:
accuracy_score(df.sentiment, df.vader_pred)

0.69556

- 예측 비교

In [94]:
cdf = df[['sentiment', 'pred', 'vader_pred']]
cdf.head(10)

,sentiment,pred,vader_pred
0,1,1,0
1,1,1,1
2,0,0,0
3,0,0,1
4,1,0,1
5,1,0,1
6,0,1,0
7,0,0,0
8,0,0,1
9,1,1,1
